In [1]:
import os
from dotenv import load_dotenv
from igdb.wrapper import IGDBWrapper
import json
import pandas as pd

load_dotenv()
ID = os.getenv('CLIENTID')
TOKEN = os.getenv('ACCESSTOKEN')

wrapper = IGDBWrapper(ID, TOKEN)

In [21]:
import time

keys = ['id', 'category', 'rating', 'rating_count', 'aggregated_rating', \
        'first_release_date', 'follows', 'game_modes', \
        'genres', 'involved_companies', 'keywords', 'summary', 'multiplayer_modes', \
        'name', 'platforms', 'player_perspectives', 'similar_games', 'themes']

batch = {k: [] for k in keys}

byte_array = wrapper.api_request(
            'games',
            'fields id, name, rating, aggregated_rating, category, \
             first_release_date, follows, rating_count, game_modes, genres, involved_companies,\
             keywords, multiplayer_modes, platforms, summary, player_perspectives,\
             similar_games, themes, expanded_games; limit 500; \
             where category = 0 & first_release_date > 1; sort first_release_date;'
          )

count = 1

while len(byte_array) > 2:
    modfields = ['game_modes', 'genres', 'platforms', 'involved_companies', 'keywords', \
                     'multiplayer_modes', 'player_perspectives', 'similar_games', 'themes']
    
    for i in json.loads(byte_array.decode('utf-8')):
        for modfield in modfields:
            try:
                i[modfield] = ' '.join([str(s) for s in i[modfield]])
            except KeyError:
                i[modfield] = ''
    
        for key in keys:
            try:
                batch[key].append(i[key])
            except KeyError:
                batch[key].append(None)
            
    time.sleep(0.25)
    
    byte_array = wrapper.api_request(
            'games',
            f'fields id, name, rating, aggregated_rating, category, \
             first_release_date, follows, rating_count, summary, game_modes, genres, involved_companies,\
             keywords, multiplayer_modes, platforms, player_perspectives,\
             similar_games, themes, expanded_games; limit 500; offset {500*count}; \
             where category = 0 & first_release_date > 1; sort first_release_date;'
          )
    count += 1


base = pd.DataFrame(batch)

In [22]:
base.head()

,id,category,rating,rating_count,aggregated_rating,first_release_date,follows,game_modes,genres,involved_companies,keywords,summary,multiplayer_modes,name,platforms,player_perspectives,similar_games,themes
0,56115,0,NaN,NaN,NaN,15552000,NaN,,5,,1 72 444 453 992 1025 1033 1208 1459 1739 2841...,"An early 3D flight simulator game, released by...",,Jet Rocket,52,1,8559 10964 11171 15536 28751 57217 58075 64311...,1
1,11304,0,NaN,NaN,NaN,21945600,NaN,1,,91161 91162,9 2396 4260,Highnoon is a BASIC game developed by Christop...,,Highnoon,82 107,5,18290 58867 62473 79064 100800 103301 103302 1...,1
2,76506,0,NaN,NaN,NaN,31449600,NaN,1,13 32,150176 150177,,"The Game of Life, also known simply as Life, i...",,Game of Life,6 69 91 103,,28277 33603 37419 44087 51577 65827 76263 7913...,
3,11485,0,NaN,NaN,50.0,47174400,NaN,1,5 15 30,50077,934 947 970 1025 8241,Star Trek is a text-based computer game that p...,,Star Trek,104 106,5,13200 13660 17130 24620 34823 55038 57187 6828...,18
4,11396,0,NaN,NaN,NaN,52531200,3.0,2,5,46322,947 970 1025 10211,Galaxy Game is one of the earliest known coin-...,,Galaxy Game,108,3,17130 24620 55038 57187 68286 68287 78770 1032...,18


In [23]:
import os.path

datasetdir = os.path.join('.', 'dataset')
filename = 'games.csv'
path = os.path.join(datasetdir, filename)
base.to_csv(path, index=False)

In [20]:
byte_array = wrapper.api_request(
            'keywords',
            'fields name; limit 500;where id = (934, 947, 970, 1025, 8241);')
for i in json.loads(byte_array.decode('utf-8')):
    print(i)

{'id': 947, 'name': 'spaceship'}
{'id': 970, 'name': 'sci-fi'}
{'id': 8241, 'name': 'discrete logic'}
{'id': 934, 'name': 'pinball'}
{'id': 1025, 'name': 'shooter'}
